In [1]:
from sklearn.model_selection import GridSearchCV # For Hyperparameter tuning and cross validation
from sklearn.linear_model import Ridge # Ridge regression model
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Load
X, y = fetch_california_housing(return_X_y=True)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the hyperparameter gris we want to search over
# 'alpha' controls the regularisation strength in Ridge Regression
# Smaller alpha = less regularisation (more flexible model)
# Larger alpha = more regularisation (simpler model)
params = {'alpha': [0.1, 1.0, 10.0]}

# Set up the Ridge regression model and wrap it in GridSearchCV
# This will try each value of alpha using a 5-fold cross-valdation
grid = GridSearchCV(Ridge(), param_grid=params, cv=5)

# Fit the model to the training data
# For each alpha, it runs 5-fold CV and scores the results
grid.fit(X_train, y_train)

# Output the best hyperparameter value found
# This tells us which alpha led to the best performance (on advage across the five folds)
print("Best alpha: ", grid.best_params_)

# Output the average cross-validation score for the best model
# This is typically Rsquared (default scoring for Ridge in regression)
print("Best score: ", grid.best_score_)



Best alpha:  {'alpha': 0.1}
Best score:  0.607159939336843


### 🔢 **1. "Should we do a confusion matrix after GridSearchCV?"**

**When doing classification (e.g. Logistic Regression):**

Yes, definitely on the test set. GridSearchCV helps us find the best model, but once we’ve selected it, we still want to evaluate its real-world performance. A confusion matrix shows exactly how your model is making mistakes: which classes are being confused.


➡️ **Use:**

```python
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test_scaled)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

```

---

### 🎯 **2. "Do we evaluate using classification_report or accuracy?"**

Both are useful—but classification_report is better.
It gives you **precision, recall, F1-score**, and support for each class, not just overall accuracy. For unbalanced datasets, F1 is a better judge of performance.


---

### ⚖️ **3. "What’s the difference between `alpha` in Ridge and `C` in LogisticRegression?"**

They're both regularization parameters—but inverse of each other:

- `alpha` (Ridge) → **higher = stronger regularization**
- `C` (LogisticRegression) → **lower = stronger regularization**

💡 So, `alpha ↑` behaves like `C ↓`.

---

### 📉 **4. "Why not use LinearRegression instead of Ridge?"**

You can, but Ridge is more stable, especially when:
- You have **many correlated features**
- You want to **avoid large, unstable coefficients**

It’s a regularised version that helps prevent **overfitting**, which LinearRegression doesn’t do.

---

### 🔍 **5. "Should we use the test set inside GridSearchCV?"**

No. Never. 
GridSearchCV only works with the **training set**, splitting it into folds internally.


You should **only test on the test set once**, at the end, to get a fair estimate of final performance.

---

### 📊 **6. "Should we use cross_val_score manually instead of GridSearchCV?"**

You could, but GridSearchCV is better for tuning because:

- It does the cross-validation **and** searches over parameter combinations automatically
- It simplifies the process
- Use `cross_val_score()` only if you want to **evaluate a fixed model**, not tune it

---

### 🧠 **7. "What’s the difference between parameters and hyperparameters?"**


- **Parameters** = learned by the model during training (e.g. weights, intercepts)
- **Hyperparameters** = set *before* training, like `C`, `alpha`, `max_depth`

GridSearchCV tunes **hyperparameters**, not model parameters.

---

### 📦 **8. "Should we use a pipeline with GridSearchCV?"**

Ideally, yes—especially if you’re scaling, doing PCA, or encoding. 

A `Pipeline` ensures all transformations (like scaling) are:

- Done in the right order
- Applied properly within each CV fold (no leakage!)

---

### 🧪 **9. "Is GridSearchCV overkill for small datasets?"**

Not really. Even on small datasets, hyperparameters can make a big difference.

But if your data is *very* small, use **fewer folds** (like `cv=3`) or just test manually.

---

### 🕵️‍♂️ **10. "How do I know if my model is overfitting or underfitting?"**

Overfitting: Very high train score, low test scoreUnderfitting: Both train and test scores are low

Check the **cross-val score vs test score**:

- If they match closely, the model generalizes well
- If they’re far apart, it's probably overfitting